In [ ]:
# Dataframes for general (pd) and geospatial data (gpd)
import pandas as pd
import geopandas as gpd

# Handling vector data
import fiona
# Handling raster data
import rasterio

In [ ]:
# To do: 
# Download MapSpam tif
# Download gridded GDP
# Download WorldPop SouthAsia
# Download groads
# Download hydrosheds watersheds tif
# Download WHYMAP groundwater vulnerability

# Upload each to S3

# Demo ogr commands for subsetting

In [ ]:
# List the RW API names of datasets desired
names_of_desired_datasets = ["MapSPAM.tif", "HydroShedsWatersheds.shp", "OSMRailways.shp"]

s3 = boto3.client('s3')

# S3 calls to download that data
for dataset in names_of_desired_datasets:
    s3.download_file("wri-projects", "resourcewatch/" + dataset, dataset)


# Create window of raster data, 
# including if there are sub-variables

# Format: [xmin ymin xmax ymax]
study_area_boundary = [0, 0, 180, 180]

with rasterio.open('mapspam.tif', 'r') as raster:

    ul = raster.index(*study_area_boundary[0:2])
    lr = raster.index(*study_area_boundary[2:4])

    window = ((lr[0], ul[0]+1), (ul[1], lr[1]+1))
    clipped_data = raster.read_band(1, window=window)
    
    # Specify which variable is wanted
    var_id = "However Rice is represented in MapSpam"
    
    clipped_filtered_data = clipped_data[clipped_data==var_id]
    

# Create window of vector data
# Decide whether to clip vectors at edges or include overspill bits

# https://gis.stackexchange.com/questions/57964/get-vector-features-inside-a-specific-extent

# with Fiona, doesnt clip polys

from shapely.geometry import mapping, shape
import fiona
# Read the original Shapefile
input = fiona.open('data.shp', 'r')
# bounds of the original shapefile
input.bounds
(258018.9133083854, 158162.863836, 268763.670357, 162621.686305)
# clip the shapefile with the raster bounds 
clipped = input.filter(bbox=((262236.3101588468, 159973.80344954136, 263491.7250217228, 160827.485556297)))
# create the clipped shapefile with the same schema
clipped_schema = input.schema.copy()
with fiona.collection('clipped.shp', 'w', 'ESRI Shapefile', clipped_schema) as output:
    for elem in clipped:
           output.write({'properties': elem['properties'],'geometry': mapping(shape(elem['geometry']))})


# With GeoPandas, does clip polys
import geopandas as gp
from shapely.geometry import Polygon

# Point to the folder containing the unzipped shapefile
basemap = gp.read_file('ne_10m_admin_0_countries')

# Set the bounds of the crop box
xmin, xmax, ymin, ymax = 6, 18, 36, 50 
bounds = Polygon( [(xmin,ymin), (xmin, ymax), (xmax, ymax), (xmax,ymin)] )

# Crop all polygons and take the part inside the bounding box
basemap['geometry'] = basemap['geometry'].intersection(bounds)

# Export non-empty geometries to shp
basemap[basemap.geometry.area>0].to_file("countries_CROPPED.shp", driver='ESRI Shapefile')



# with ogr2ogr
# -spat xmin ymin xmax ymax and -clipsrc [xmin ymin xmax ymax]
# http://www.gdal.org/ogr2ogr.html
# https://gis.stackexchange.com/questions/59526/ogr2ogr-extract-shp-file-by-extent-with-bash-script
# https://anitagraser.com/2012/05/09/batch-shapefile-clipping/

### Q: how does this handle zipped files?
# Yes: https://github.com/dwtkns/gdal-cheat-sheet
# Read from a zip file
# This assumes that archive.zip is in the current directory. 
# This example just extracts the file, but any ogr2ogr operation should work. 
# It's also possible to write to existing zip files.
# ogr2ogr -f 'GeoJSON' dest.geojson /vsizip/archive.zip/zipped_dir/in.geojson

# Set env variable
os.environ["xmin"] = bdy[0]
os.environ["ymin"] = bdy[1]
os.environ["xmax"] = bdy[2]
os.environ["ymax"] = bdy[3]

# Equivalent to fiona approach
!ogr2ogr -spat $xmin $ymin $xmax $ymax output.shp /vsizip/$inputzip/zipped_dir/input.shp

# Equivalent to geopandas approach
!ogr2ogr -clipsrc [$xmin $ymin $xmax $ymax ] output.shp /vsizip/$inputzip/zipped_dir/input.shp



In [ ]:
bdy = [1,2,3,4]
print(*bdy[0:2])